In [50]:
# import data
import numpy as np
import pandas as pd
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

#### Utils


In [51]:
# Validate df object
# ---------------------------------------------------------------------------------#
def isDFThrow(df: pd.DataFrame):
    """Check if the input is a Pandas DataFrame, if not, raise ValueError."""
    if not isinstance(df, pd.DataFrame):
        raise ValueError("df is not a DataFrame")


# Move DF columns
# ---------------------------------------------------------------------------------#


def move_columns_to_end_by_len(df: pd.DataFrame, num_columns: int) -> pd.DataFrame:
    # Revisar si el número de columnas a mover es válido
    if num_columns >= len(df.columns):
        print(
            "El número de columnas a mover excede el número total de columnas en el DataFrame."
        )
        return df

    # Reordenar las columnas
    cols = list(df.columns)
    cols = cols[num_columns:] + cols[:num_columns]
    df = df[cols]

    return df


# Comparar igualdad de decimales midiendo la precision decimal
# ---------------------------------------------------------------------------------#
def media_precision_decimales_by_index(df1, df2, indices):
    precision_decimales = []

    # Iterar sobre los índices proporcionados
    for idx in indices:
        # Obtener las filas correspondientes en los DataFrames
        fila1 = df1.loc[idx]
        fila2 = df2.loc[idx]

        # Iterar sobre las columnas numéricas
        for col in df1.select_dtypes(include=["float64"]).columns:
            # Comprobar si los valores de los decimales son iguales
            if fila1[col] == fila2[col]:
                # Calcular la precisión de los decimales
                precision_decimales.append(len(str(fila1[col]).split(".")[1]))

    # Calcular la media de la precisión de los decimales
    if precision_decimales:
        media = sum(precision_decimales) / len(precision_decimales)
    else:
        media = 0

    return media

#### Estratificado de datos

- [ejecución paso a paso Code](/task/pia04-task/functions/splitDataToTrain.py)


In [52]:
def SplitDataToTrain(df: pd.DataFrame) -> pd.DataFrame:
    try:
        isDFThrow(df)

        ###  split data to train
        ## split data to train
        X = df.drop(columns="median_house_value")
        y = df["median_house_value"]

        ### Muestreo estratificado (*Stratified sampling*)
        ### dividiendo el *dataset* en grupos llamados **estratos**,
        ### y asegurándose de tomar no solo un porcentaje de muestras del total,
        ### sino ese porcentaje de cada estrato.
        stratify = pd.cut(
            df["median_income"],
            bins=[
                0.0,
                1.5,
                3.0,
                4.5,
                6.0,
                np.inf,
            ],  # Secuencia de límites de los contenedores
            labels=[1, 2, 3, 4, 5],  # dividimos en 5 categorías
        )

        # https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
        ## objetivo --> evitar el **sobre-ajuste** o ***over-fitting***
        ##  X_train, X_test, y_train, y_test = train_test_split(
        return train_test_split(
            X,  # features
            y,  # target
            stratify=stratify,  # estratificado
            test_size=0.2,  #  estoy usando el 20% de los datos # Controla la mezcla aplicada a los datos antes de aplicar la división.
            random_state=42,  ## fixed seed
        )
    except ValueError as ve:
        # Manejo de la excepción
        print("ValueError:", ve)

#### Ejecución Secuencial

- [ejecución paso a paso NoteBook](/task/pia04-task/pia04-task-sequential.ipynb)
- [ejecución paso a paso Code](/task/pia04-task/functions/firstOheSequential.py)


In [53]:
def FirstOheSequential(df: pd.DataFrame) -> pd.DataFrame:
    try:
        isDFThrow(df)
        # dfVerifiqueNullAndNotNumberValues(df)
        # columnsWithNonNumericValues = dfGetColumnsWithNonNumericValues(df)
        # columnsWithNullValues = dfGetColumnsWithNullValues(df)
        ## para comprobar la efectividad de las operaciones que vamos a realizar
        ### obtenemos los índices de las filas con valores nulos
        null_rows_idx = df.isnull().any(axis=1)

        ## OHE
        ### Aplicamos también OneHotEncoder para las variables categóricas.
        cat_encoder = OneHotEncoder(sparse_output=False).set_output(
            transform="pandas"
        )  # forzamos que la salida sea DataFrame
        X_train_ocean_proximity_ohe = cat_encoder.fit_transform(df[["ocean_proximity"]])

        ### recuperar el resto de DataFrame
        X_train_rest = df.drop(columns="ocean_proximity")
        data_cat_ohe = pd.concat([X_train_rest, X_train_ocean_proximity_ohe], axis=1)

        ## Estandarización
        scaler = StandardScaler().set_output(
            transform="pandas"
        )  # Para que el resultado sea un DataFrame
        X_train_scaled_and_ohe = scaler.fit_transform(data_cat_ohe)
        # p("X_train_scaled_and_ohe", X_train_scaled_and_ohe)

        k_value = np.sqrt(df.shape[0]).astype(int)

        ## Imputamos valores
        X_train_imputed_a = (
            KNNImputer(n_neighbors=k_value)
            .set_output(transform="pandas")
            .fit_transform(X_train_scaled_and_ohe)
        )

        ## Verificamos la actualización de los valores nulos
        X_train_imputed_a.loc[
            null_rows_idx
        ].head()  # visualizamos las filas que tenían valores nulos

        return pd.DataFrame(X_train_imputed_a)

    except ValueError as ve:
        # Manejo de la excepción
        print("ValueError:", ve)

#### Ejecución mediante PipineLine

- [ejecución paso a paso NoteBook](/task/pia04-task/pia04-task-pipe.ipynb)
- [ejecución paso a paso Code](/task/pia04-task/functions/firstOhePipe.py)


In [54]:
def FirstOhePipe(df: pd.DataFrame) -> pd.DataFrame:
    try:
        # comprobaciones
        isDFThrow(df)
        # definimos variables
        k_value = int(np.sqrt(df.shape[0]))
        # Identificar las columnas categóricas y numéricas
        categorical_columns = df.select_dtypes(include=["object"]).columns
        numerical_columns = df.select_dtypes(include=["float64", "int64"]).columns

        # Crear un Pipeline
        ##  1. ejecute OHE - OneHotEncoder
        ###  - dtype_include="object" --> aplicamos solamente a datos no numericos
        ###  -  remainder="passthrough" --> tras aplicar los cambios necesitamos recuperar el resto de columnas
        ##  2. estandarice - StandardScaler
        ##  3. impute los datos nulos o faltantes - KNNImputer
        pipeline = make_pipeline(
            make_column_transformer(
                (OneHotEncoder(), make_column_selector(dtype_include="object")),
                remainder="passthrough",
            ),
            StandardScaler(),
            KNNImputer(n_neighbors=k_value),
        )
        # Aplicamos las trasformaciones y las imputaciones al DataFrame
        array_transformed = pipeline.fit_transform(df)
        # Convertir el resultado imputado de nuevo a un DataFrame
        ## Necesitamos obtener los nombres de las nuevas columnas después de OHE y el resto
        ohe_columns = (
            pipeline.named_steps["columntransformer"]
            .named_transformers_["onehotencoder"]
            .get_feature_names_out(input_features=categorical_columns)
        )
        all_columns = np.concatenate([ohe_columns, numerical_columns])
        # Generamos un nuevo DF
        df_transformed = pd.DataFrame(array_transformed, columns=all_columns)
        # nota las columnas ohe se colocan al principio del df, las ponemos al final
        # para que coincidan con eñ resultado secuencial
        return move_columns_to_end_by_len(df_transformed, len(ohe_columns))

    except ValueError as ve:
        # Manejo de la excepción
        print("ValueError:", ve)

#### Comparación de los resultados

- [ejecución paso a paso NoteBook](/task/pia04-task/pia04-task-compare.ipynb)
- [ejecución paso a paso Code](/task/pia04-task/functions/dfCompare.py)


In [55]:
def DFCompare(
    df1: pd.DataFrame,
    df2: pd.DataFrame,
) -> dict:
    res = {}

    # ❗Important to compare first reset index
    df1 = df1.reset_index(drop=True)
    df2 = df2.reset_index(drop=True)

    res["df_size_eq"] = df1.shape == df2.shape
    res["df_data_type_eq"] = df1.dtypes.equals(df2.dtypes)
    res["df_columns_eq"] = df1.columns.equals(df2.columns)
    res["df_rows_eq"] = df1.index.equals(df2.index)
    res["df_value_00_eq"] = (df1.iloc[0, 0]) == (df2.iloc[0, 0])
    res["df_value_00_eq_round8"] = (df1.iloc[0, 0].round(8)) == (
        df2.iloc[0, 0].round(8)
    )
    res["df_values_eq"] = (df1 == df2).all().all()
    res["df_values_eq_round8"] = (df1.round(8) == df2.round(8)).all().all()
    return res

#### Main ejecución del Programa

- [ejecución paso a paso Code](/task/pia04-task/main.py)


In [56]:
df = pd.read_csv("./data/housing.csv")

print(" == Step1 Split Data to Train ==")
X_train, X_test, y_train, y_test = SplitDataToTrain(df)
null_rows_idx = df.isnull().any(axis=1)
print(" == Step2 OHE sequential ==")
ohe_sequential = FirstOheSequential(X_train.copy())
print(ohe_sequential[ohe_sequential.columns[-5:]].head(3).T)
print(" == Step3 OHE pipe ==")
ohe_pipe = FirstOhePipe(X_train.copy())
print(ohe_pipe[ohe_pipe.columns[-5:]].head(3).T)
# onehotencoder_column_index = [1,2,3,4]
print(" == Step4 Compare OHE sequential vs OHE pipe ==")
compare = DFCompare(ohe_sequential, ohe_pipe)
for key, value in compare.items():
    print(key, ":", value)

 == Step1 Split Data to Train ==
 == Step2 OHE sequential ==
                               12655     15502     2908 
ocean_proximity_<1H OCEAN  -0.887683 -0.887683 -0.887683
ocean_proximity_INLAND      1.462180 -0.683910  1.462180
ocean_proximity_ISLAND     -0.011006 -0.011006 -0.011006
ocean_proximity_NEAR BAY   -0.354889 -0.354889 -0.354889
ocean_proximity_NEAR OCEAN -0.384217  2.602693 -0.384217
 == Step3 OHE pipe ==
                                   0         1         2
ocean_proximity_<1H OCEAN  -0.887683 -0.887683 -0.887683
ocean_proximity_INLAND      1.462180 -0.683910  1.462180
ocean_proximity_ISLAND     -0.011006 -0.011006 -0.011006
ocean_proximity_NEAR BAY   -0.354889 -0.354889 -0.354889
ocean_proximity_NEAR OCEAN -0.384217  2.602693 -0.384217
 == Step4 Compare OHE sequential vs OHE pipe ==
df_size_eq : True
df_data_type_eq : True
df_columns_eq : True
df_rows_eq : True
df_value_00_eq : False
df_value_00_eq_round8 : True
df_values_eq : False
df_values_eq_round8 : True


<style>
dt {
  color:blue;
  font-weight: bold;
}
dt {

}
</style>
<dl>
<dt>train_test_split</dt>
<dd>This is the definition of the first term.</dd>
<dt>OneHotEncoder</dt>
<dd>This is one definition of the second term. </dd>
<dt>StandardScaler</dt>
<dd>This is one definition of the second term. </dd>
<dt>KNNImputer(n_neighbors=k_value)</dt>
<dd>This is one definition of the second term. </dd>
<dt>make_pipeline</dt>
<dd>This is one definition of the second term. </dd>
<dt>make_column_transformer</dt>
<dd>This is one definition of the second term. </dd>
<dd>atributo: remainder="passthrough"</dd>
</dl>
